# UltimateGuitarTabs Scraper

The following notebook scrapes the top 1000 most popular tabs posting on https://www.ultimate-guitar.com/

After scraping, it stores the data found into a SQLite3 database ('../data/UltimateGuitarTabs.db') with the following tables:
1. Tab_Data (id, song, artist, is_acoustic, tab_url)
    - id: Unique tab ID 
    - song: Name of song
    - artist: Name of artist
    - is_acoustic: Song is played acoustically (1), otherwise (0)
    - tab_url: URL to tab
2. Artists (name, url)
    - name: Name of artist
    - url: URL to artist profile
3. Hits (id, num_hits, votes, rating)
    - id: Unique tab ID
    - num_hits: # of times tab visited
    - votes: # number of votes received
    - rating: Avg rating out of 5
4. Chords (song, artist, tonality, capo, chords)
    - song: Name of song
    - artist: Name of artist
    - tonality: Original of song
    - capo: Fret to place capo
    - chords: String of full chord progression

Originally scraped on June 5, 2018
Rescraped on March 5, 2021

In [1]:
import sqlite3
import re
import json
import pandas as pd
import time
from bs4 import BeautifulSoup
from requests import get
from requests.exceptions import RequestException
from contextlib import closing

In [2]:
def simple_get(url):
    """
    Source: https://realpython.com/python-web-scraping-practical-introduction/
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Source: https://realpython.com/python-web-scraping-practical-introduction/
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    Source: https://realpython.com/python-web-scraping-practical-introduction/
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

def parse_data_content_from_url(url, max_retries=5):
    """
    This function parses the content on the given URL webpage and grabs the "data-content" 
    from the <div class='js-store'> tag. The data stored in this object includes all content 
    hosted on the webpage.
    """
    html = ''
    soup = ''
    attempt_num = 1
    while (html == '' or soup == '') and attempt_num < max_retries:
        try:
            print('Attempting to reach link: {}'.format(url))
            html = simple_get(url)
            soup = BeautifulSoup(html, 'html.parser')
            data_content = json.loads(soup.find("div", {'class':'js-store'})['data-content'])
            print('Succesfully loaded <div data-content>')
            return(data_content)
        except:
            attempt_num+=1
            print('Connection refused on server for link: {}'.format(url))
            print('Retrying for attempt {0}/{1} in 5 seconds'.format(attempt_num, max_retries))
            time.sleep(5)
            continue        
            
            
def parse_chords_from_url(url):
    """
    Scrapes and returns the sequences of 
    chords as a list as well as the fret number
    to place a capo. 
    """

    data = parse_data_content_from_url(url)

    chords = data['store']['page']['data']['tab_view']['wiki_tab']['content']
    
    # Matching groups (open tag)(chord pitch)(base note {0 or 1})(chord type)(base note {0 or 1})(closing tag)
    pattern = "(\[ch\])([A-G]+)(\/[A-G]*[b#])*([(?m)|(?m\d)|(?b\d)|(?#\d)|(?maj\d)|(?add\d)|(?sus\d)|(?aug)|(?aug\d)|(?dim)|(?dim\d)]*)(\/[A-G]*[b#])*(\[\/ch\])"
    prog = re.compile(pattern)
    result = prog.findall(chords)
    
    cleaned_res = result
    for i in range(len(result)):
        # Grabbing groups (chord pitch)(base note)(chord type)(base note)
        cleaned_res[i] = result[i][1] + result[i][2] + result[i][3] + result[i][4]
       
    # Grabbing Capo info
    capo = 0
    try:
        capo = data['store']['page']['data']['tab_view']['meta']['capo']
    except:
        capo = 0
        
    return(cleaned_res, capo)
            
def get_metadata_from_top_page(url):
    data_content = parse_data_content_from_url(url)
    tabs = data_content['store']['page']['data']['data']['tabs']
    hits = data_content['store']['page']['data']['data']['hits']
    
    return(tabs, hits)    

def get_multiple_pages(url, n):
    """
    Creates functionality to scrape multiple
    pages up to n
    """
    page_suffix = "&page="
    tabs_list = []
    hits_list = []
    
    for i in range(n):
        cur_tabs, cur_hits = get_metadata_from_top_page(url + page_suffix + str(i+1))
        
        tabs_list += cur_tabs
        hits_list += cur_hits
        
    return(tabs_list, hits_list)

def parse_tab_fields(tab_obj, hit_obj):
    tab_url = tab_obj['tab_url']
    chords, capo = parse_chords_from_url(tab_url)
    tab_dict = {
        'tab_id': int(tab_obj['id']),
        'song_name': re.sub("'", "", tab_obj['song_name']),
        'artist': re.sub("'", "", tab_obj['artist_name']),
        'tonality': tab_obj['tonality_name'],
        'votes': int(tab_obj['votes']),
        'rating': float(tab_obj['rating']),
        'is_acoustic': int(tab_obj['recording']['is_acoustic']),
        'tab_url': tab_obj['tab_url'],
        'artist_url': tab_obj['artist_url'],
        'hit_id': int(hit_obj['id']),
        'hit_num': int(hit_obj['hits']),
        'chords': ','.join(chords),
        'capo': capo
    }
    return tab_dict

def create_sqlite_connector(db='UltimateGuitarTabs.db'):
    con = sqlite3.connect('../data/{}'.format(db))
    cur = con.cursor()
    return(con, cur)

def drop_tables_if_exists(con, cur):
    delete_tabs = "DROP TABLE IF EXISTS Tab_Data;"
    cur.execute(delete_tabs)
    con.commit()
    
    delete_artists = "DROP TABLE IF EXISTS Artists;"
    cur.execute(delete_artists)
    con.commit()
    
    delete_hits = "DROP TABLE IF EXISTS Hits;"
    cur.execute(delete_hits)
    con.commit()
    
    delete_chords = "DROP TABLE IF EXISTS Chords;"
    cur.execute(delete_chords)
    con.commit()
    
def create_tables(con, cur):
    create_tabs = """
        CREATE TABLE Tab_Data (
            id integer primary key,
            song text,
            artist text,
            is_acoustic integer,
            tab_url text
        )"""
    cur.execute(create_tabs)
    con.commit()    

    create_artists = """
        CREATE TABLE Artists (
            name text,
            url text
        )"""
    cur.execute(create_artists)
    con.commit()


    create_hits = """
        CREATE TABLE Hits (
            id integer primary key, 
            num_hits integer, 
            votes integer,
            rating float
        )"""
    cur.execute(create_hits)
    con.commit()    

    create_chords = """
        CREATE TABLE Chords (
        id integer primary key,
        song text,
        artist text,
        tonality text,
        capo integer,
        chords text
        )"""
    cur.execute(create_chords)  
    con.commit()
    
def insert_data(tab_dict, con, cur):
    try: 
        sql_tab = "insert into Tab_Data (id,song,artist,is_acoustic,tab_url) \
        VALUES ('%d','%s','%s','%d','%s')" % \
            (tab_dict['tab_id'], tab_dict['song_name'], tab_dict['artist'],
             tab_dict['is_acoustic'], tab_dict['tab_url'])
        cur.execute(sql_tab)
        con.commit()

    except:
        print('Unable to insert into Tab_Data')
        print(tab_dict)        
        
    try:
        sql_artist = "insert into Artists (name, url) VALUES ('%s','%s')" %\
            (tab_dict['artist'], tab_dict['artist_url'])
        cur.execute(sql_artist)
        con.commit()
    except:
        print('Unable to insert into Artists')
        print(tab_dict)
        

    try:
        sql_hit = "insert into Hits (id, num_hits, votes, rating) VALUES ('%d','%d','%d','%.8f')" %\
            (tab_dict['tab_id'], tab_dict['hit_num'],
             tab_dict['votes'], tab_dict['rating'])
        cur.execute(sql_hit)
        con.commit()
        
    except:
        print('Unable to insert into Hits')
        print(tab_dict)
    
    try:
        sql_chords = "insert into Chords (id, song, artist, tonality, capo, chords) VALUES('%d','%s','%s','%s','%d','%s')" % \
            (tab_dict['tab_id'], tab_dict['song_name'],
             tab_dict['artist'], tab_dict['tonality'],
             int(tab_dict['capo']),
             tab_dict['chords'])
        cur.execute(sql_chords)
        con.commit()
        
    except:
        print('Unable to insert into Chords')
        print(tab_dict)

    
def scrape_ultimate_guitar(url='https://www.ultimate-guitar.com/explore?order=hitstotal_desc&type[]=Chords', pages=20):
    con, cur = create_sqlite_connector()
    
    print('Dropping sqlite tables')
    drop_tables_if_exists(con, cur)
    
    print('Creating sqlite tables')
    create_tables(con, cur) 

    print('Parsing pages')
    tabs_list, hits_list = get_multiple_pages('https://www.ultimate-guitar.com/explore?order=hitstotal_desc&type[]=Chords', 20)    
    
    print('Parsing tabs for each page')
    for i in range(len(tabs_list)):
        print('Parsing tab #{}'.format(i))
        tab_dict = parse_tab_fields(tabs_list[i], hits_list[i])
        insert_data(tab_dict, con, cur)
        
    con.close()    

In [3]:
scrape_ultimate_guitar()

Dropping sqlite tables
Creating sqlite tables
Parsing pages
Attempting to reach link: https://www.ultimate-guitar.com/explore?order=hitstotal_desc&type[]=Chords&page=1
Succesfully loaded <div data-content>
Attempting to reach link: https://www.ultimate-guitar.com/explore?order=hitstotal_desc&type[]=Chords&page=2
Succesfully loaded <div data-content>
Attempting to reach link: https://www.ultimate-guitar.com/explore?order=hitstotal_desc&type[]=Chords&page=3
Succesfully loaded <div data-content>
Attempting to reach link: https://www.ultimate-guitar.com/explore?order=hitstotal_desc&type[]=Chords&page=4
Succesfully loaded <div data-content>
Attempting to reach link: https://www.ultimate-guitar.com/explore?order=hitstotal_desc&type[]=Chords&page=5
Succesfully loaded <div data-content>
Attempting to reach link: https://www.ultimate-guitar.com/explore?order=hitstotal_desc&type[]=Chords&page=6
Succesfully loaded <div data-content>
Attempting to reach link: https://www.ultimate-guitar.com/explor

Succesfully loaded <div data-content>
Parsing tab #34
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/johnny-cash/hurt-chords-89849
Succesfully loaded <div data-content>
Parsing tab #35
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/bruno-mars/just-the-way-you-are-chords-970416
Succesfully loaded <div data-content>
Parsing tab #36
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/john-lennon/imagine-chords-9306
Succesfully loaded <div data-content>
Parsing tab #37
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/eagles/hotel-california-chords-46190
Succesfully loaded <div data-content>
Parsing tab #38
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/ed-sheeran/the-a-team-chords-989712
Succesfully loaded <div data-content>
Parsing tab #39
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/onerepublic/counting-stars-chords-1233464
Succesfully loaded <div data-content>
Parsing tab #40
Attempting to reach l

Succesfully loaded <div data-content>
Parsing tab #86
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/shawn-mendes/stitches-chords-1721601
Succesfully loaded <div data-content>
Parsing tab #87
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/train/drops-of-jupiter-chords-1433
Succesfully loaded <div data-content>
Parsing tab #88
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/cat-stevens/father-and-son-chords-84491
Succesfully loaded <div data-content>
Parsing tab #89
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/radiohead/karma-police-chords-103398
Succesfully loaded <div data-content>
Parsing tab #90
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/bob-marley/redemption-song-chords-417640
Succesfully loaded <div data-content>
Parsing tab #91
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/4-non-blondes/whats-up-chords-349210
Succesfully loaded <div data-content>
Parsing tab #92
Attempting to rea

Succesfully loaded <div data-content>
Parsing tab #139
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/aerosmith/i-dont-want-to-miss-a-thing-chords-59465
Succesfully loaded <div data-content>
Parsing tab #140
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/john-denver/take-me-home-country-roads-chords-57606
Succesfully loaded <div data-content>
Parsing tab #141
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/hozier/take-me-to-church-chords-1442757
Succesfully loaded <div data-content>
Parsing tab #142
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/bruno-mars/the-lazy-song-chords-992284
Succesfully loaded <div data-content>
Parsing tab #143
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/jason-mraz/im-yours-chords-499752
Succesfully loaded <div data-content>
Parsing tab #144
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/taylor-swift/blank-space-chords-1674053
Succesfully loaded <div data-content

Succesfully loaded <div data-content>
Parsing tab #191
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/poison/every-rose-has-its-thorn-chords-130272
Succesfully loaded <div data-content>
Parsing tab #192
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/adele/make-you-feel-my-love-chords-838744
Succesfully loaded <div data-content>
Parsing tab #193
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/the-beatles/across-the-universe-chords-202167
Succesfully loaded <div data-content>
Parsing tab #194
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/misc-soundtrack/a-star-is-born-always-remember-us-this-way-chords-2484211
Succesfully loaded <div data-content>
Parsing tab #195
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/coldplay/fix-you-chords-1863305
Succesfully loaded <div data-content>
Parsing tab #196
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/natalie-imbruglia/torn-chords-11489
Succesfully load

...

Succesfully loaded <div data-content>
Parsing tab #763
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/the-chainsmokers/dont-let-me-down-chords-1818535
Succesfully loaded <div data-content>
Parsing tab #764
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/john-denver/annies-song-chords-38904
Succesfully loaded <div data-content>
Parsing tab #765
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/maroon-5/payphone-chords-1148461
Succesfully loaded <div data-content>
Parsing tab #766
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/elvis-presley/cant-help-falling-in-love-chords-43273
Succesfully loaded <div data-content>
Parsing tab #767
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/the-killers/human-chords-741479
Succesfully loaded <div data-content>
Parsing tab #768
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/the-beatles/twist-and-shout-chords-489173
Succesfully loaded <div data-content>
Parsing 

Succesfully loaded <div data-content>
Parsing tab #815
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/chris-tomlin/our-god-chords-910102
Succesfully loaded <div data-content>
Parsing tab #816
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/david-cook/always-be-my-baby-chords-672972
Succesfully loaded <div data-content>
Parsing tab #817
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/u2/i-still-havent-found-what-im-looking-for-chords-57139
Succesfully loaded <div data-content>
Parsing tab #818
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/weezer/island-in-the-sun-chords-852868
Succesfully loaded <div data-content>
Parsing tab #819
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/michael-jackson/billie-jean-chords-258773
Succesfully loaded <div data-content>
Parsing tab #820
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/andra-day/rise-up-chords-1763927
Succesfully loaded <div data-content>
Parsi

Succesfully loaded <div data-content>
Parsing tab #867
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/the-rolling-stones/you-cant-always-get-what-you-want-chords-4019
Succesfully loaded <div data-content>
Parsing tab #868
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/taylor-swift/ours-chords-998991
Succesfully loaded <div data-content>
Parsing tab #869
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/justin-bieber/mistletoe-chords-1101293
Succesfully loaded <div data-content>
Parsing tab #870
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/cee-lo-green/forget-you-chords-1031317
Succesfully loaded <div data-content>
Parsing tab #871
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/ron-pope/a-drop-in-the-ocean-chords-1004711
Succesfully loaded <div data-content>
Parsing tab #872
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/america/sister-golden-hair-chords-39425
Succesfully loaded <div data-cont

Succesfully loaded <div data-content>
Parsing tab #919
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/adele/chasing-pavements-chords-653316
Succesfully loaded <div data-content>
Parsing tab #920
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/the-beatles/dont-let-me-down-chords-17244
Succesfully loaded <div data-content>
Parsing tab #921
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/sufian-suhaimi/di-matamu-chords-2350085
Succesfully loaded <div data-content>
Parsing tab #922
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/the-smiths/please-please-please-let-me-get-what-i-want-chords-453858
Succesfully loaded <div data-content>
Parsing tab #923
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/robbie-williams/angels-chords-300
Succesfully loaded <div data-content>
Parsing tab #924
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/khalid/location-chords-1952459
Succesfully loaded <div data-content>


Succesfully loaded <div data-content>
Parsing tab #972
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/vanessa-williams/colors-of-the-wind-chords-205250
Succesfully loaded <div data-content>
Parsing tab #973
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/lady-gaga/poker-face-chords-810048
Succesfully loaded <div data-content>
Parsing tab #974
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/rihanna/fourfiveseconds-chords-1705925
Succesfully loaded <div data-content>
Parsing tab #975
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/david-bowie/heroes-chords-1046436
Succesfully loaded <div data-content>
Parsing tab #976
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/the-temptations/my-girl-chords-86366
Succesfully loaded <div data-content>
Parsing tab #977
Attempting to reach link: https://tabs.ultimate-guitar.com/tab/3-doors-down/here-without-you-chords-107820
Succesfully loaded <div data-content>
Parsing tab #97